In [9]:
import mysql.connector
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random
from io import StringIO 


In [ ]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'https://www.google.com/',
}

years = list(range(2000, 2026))
dfs = {}

for year in years:
    print(f"Scraping season {year}...")
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"
    time.sleep(random.uniform(2, 5))

    try:
        response = requests.get(url, headers=headers)

        if response.status_code == 403:
            print(f"Blocked on first attempt for {year}, retrying...")
            headers['User-Agent'] = f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{random.randint(100,125)}.0.0.0 Safari/537.36"
            response = requests.get(url, headers=headers)
            response.encoding = 'utf-8'

        response.raise_for_status()
        html = response.content.decode('utf-8', errors='replace')
        soup = BeautifulSoup(html, 'html.parser')

        table = soup.find(name='table', attrs={'id': 'per_game_stats'})

        if table:
            df = pd.read_html(StringIO(str(table)))[0] 
            df = df[df.Player != 'Player']
            df = df.dropna(subset=['Player'])
            df['Season'] = f"{year-1}-{str(year)[2:]}"
            dfs[year] = df
            print(f"Collected {len(df)} player rows for {year}")
        else:
            print(f"No table found for season {year}")

    except Exception as e:
        print(f"Error scraping {year}: {e}")


Scraping season 2000...
Collected 497 player rows for 2000
Scraping season 2001...
Collected 538 player rows for 2001
Scraping season 2002...
Collected 501 player rows for 2002
Scraping season 2003...
Collected 484 player rows for 2003
Scraping season 2004...
Collected 586 player rows for 2004
Scraping season 2005...
Collected 586 player rows for 2005
Scraping season 2006...
Collected 564 player rows for 2006
Scraping season 2007...
Collected 517 player rows for 2007
Scraping season 2008...
Collected 596 player rows for 2008
Scraping season 2009...
Collected 583 player rows for 2009
Scraping season 2010...
Collected 579 player rows for 2010
Scraping season 2011...
Collected 626 player rows for 2011
Scraping season 2012...
Collected 552 player rows for 2012
Scraping season 2013...
Collected 574 player rows for 2013
Scraping season 2014...
Collected 612 player rows for 2014
Scraping season 2015...
Collected 652 player rows for 2015
Scraping season 2016...
Collected 579 player rows for 20

In [3]:
for years,df in dfs.items():
    df['is_total_row'] = df['Team'] == '2TM'

In [4]:
##RENAMING THE COLUMN NAMES FOR EASY SQL ENTRY
column_rename_map = {
    'Rk': 'rk',
    'Player': 'player',
    'Age': 'age',
    'Team': 'team',
    'Pos': 'pos',
    'G': 'games_played',
    'GS': 'games_started',
    'MP': 'minutes_played',
    'FG': 'fg',
    'FGA': 'fga',
    'FG%': 'fg_pct',
    '3P': 'three_pt',
    '3PA': 'three_pt_att',
    '3P%': 'three_pt_pct',
    '2P': 'two_pt',
    '2PA': 'two_pt_att',
    '2P%': 'two_pt_pct',
    'eFG%': 'efg_pct',
    'FT': 'ft',
    'FTA': 'fta',
    'FT%': 'ft_pct',
    'ORB': 'orb',
    'DRB': 'drb',
    'TRB': 'trb',
    'AST': 'ast',
    'STL': 'stl',
    'BLK': 'blk',
    'TOV': 'tov',
    'PF': 'pf',
    'PTS': 'pts',
    'Awards': 'awards',
    'Season': 'season',
    'is_total_row': 'is_total_row'
}

for year, df in dfs.items():
    dfs[year] = df.rename(columns=column_rename_map)


In [5]:
columns = ['rk', 'player', 'age', 'team', 'pos', 'games_played', 'games_started',
       'minutes_played', 'fg', 'fga', 'fg_pct', 'three_pt', 'three_pt_att',
       'three_pt_pct', 'two_pt', 'two_pt_att', 'two_pt_pct', 'efg_pct', 'ft',
       'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'awards', 'season', 'is_total_row']

In [6]:
pd.set_option('display.max_columns', None)
dfs[2025].head(10)

,rk,player,age,team,pos,games_played,games_started,minutes_played,fg,fga,fg_pct,three_pt,three_pt_att,three_pt_pct,two_pt,two_pt_att,two_pt_pct,efg_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,awards,season,is_total_row
0,1.0,Shai Gilgeous-Alexander,26.0,OKC,PG,76.0,76.0,34.2,11.3,21.8,0.519,2.1,5.7,0.375,9.2,16.1,0.571,0.569,7.9,8.8,0.898,0.9,4.1,5.0,6.4,1.7,1.0,2.4,2.2,32.7,"MVP-1,DPOY-10,CPOY-8,AS,NBA1",2024-25,False
1,2.0,Giannis Antetokounmpo,30.0,MIL,PF,67.0,67.0,34.2,11.8,19.7,0.601,0.2,0.9,0.222,11.6,18.7,0.620,0.607,6.5,10.6,0.617,2.2,9.7,11.9,6.5,0.9,1.2,3.1,2.3,30.4,"MVP-3,DPOY-8,AS,NBA1",2024-25,False
2,3.0,Nikola Jokić,29.0,DEN,C,70.0,70.0,36.7,11.2,19.5,0.576,2.0,4.7,0.417,9.3,14.8,0.627,0.627,5.2,6.4,0.800,2.9,9.9,12.7,10.2,1.8,0.6,3.3,2.3,29.6,"MVP-2,CPOY-2,AS,NBA1",2024-25,False
3,4.0,Luka Dončić,25.0,2TM,PG,50.0,50.0,35.4,9.2,20.5,0.450,3.5,9.6,0.368,5.7,10.9,0.522,0.536,6.2,7.9,0.782,0.8,7.4,8.2,7.7,1.8,0.4,3.6,2.5,28.2,NaN,2024-25,True
4,4.0,Luka Dončić,25.0,DAL,PG,22.0,22.0,35.7,9.8,21.2,0.464,3.4,9.6,0.354,6.4,11.5,0.555,0.544,5.1,6.6,0.767,0.7,7.6,8.3,7.8,2.0,0.4,3.4,2.6,28.1,NaN,2024-25,False
5,4.0,Luka Dončić,25.0,LAL,PG,28.0,28.0,35.1,8.8,20.0,0.438,3.6,9.6,0.379,5.1,10.4,0.493,0.530,7.0,8.9,0.791,0.9,7.2,8.1,7.5,1.6,0.4,3.7,2.4,28.2,NaN,2024-25,False
6,5.0,Anthony Edwards,23.0,MIN,SG,79.0,79.0,36.3,9.1,20.4,0.447,4.1,10.3,0.395,5.1,10.1,0.501,0.547,5.3,6.3,0.837,0.8,4.9,5.7,4.5,1.2,0.6,3.2,1.9,27.6,"MVP-7,CPOY-3,AS,NBA2",2024-25,False
7,6.0,Jayson Tatum,26.0,BOS,PF,72.0,72.0,36.4,9.2,20.3,0.452,3.5,10.1,0.343,5.7,10.2,0.559,0.537,5.0,6.1,0.814,0.7,8.0,8.7,6.0,1.1,0.5,2.9,2.2,26.8,"MVP-4,CPOY-10,AS,NBA1",2024-25,False
8,7.0,Kevin Durant,36.0,PHO,PF,62.0,62.0,36.5,9.5,18.1,0.527,2.6,6.0,0.430,7.0,12.1,0.574,0.598,4.9,5.8,0.839,0.4,5.7,6.0,4.2,0.8,1.2,3.1,1.7,26.6,AS,2024-25,False
9,8.0,Tyrese Maxey,24.0,PHI,PG,52.0,52.0,37.7,9.2,21.0,0.437,3.1,9.2,0.337,6.1,11.8,0.515,0.511,4.9,5.6,0.879,0.3,3.1,3.3,6.1,1.8,0.4,2.4,2.2,26.3,CPOY-10,2024-25,False


In [ ]:
CREATE TABLE players (
    playerID INT NOT NULL AUTO_INCREMENT,
    playerName VARCHAR(100) NOT NULL,
    PRIMARY KEY (playerID)
);

CREATE TABLE seasons (
    seasonID INT NOT NULL AUTO_INCREMENT,
    year VARCHAR(25) NOT NULL,
    PRIMARY KEY (seasonID)
);

CREATE TABLE teams (
    teamCode VARCHAR(5) PRIMARY KEY,
    teamName VARCHAR(100),
    franchiseID VARCHAR(10)
);

CREATE TABLE player_stats (
    playerID INT NOT NULL,
    seasonID INT NOT NULL,
    age INT NOT NULL,
    teamCode VARCHAR(5),
    pos VARCHAR(25),
    games_played INT,
    games_started INT,
    minutes_played FLOAT,
    fg FLOAT,
    fga FLOAT,
    fg_pct DECIMAL(5,3),
    three_pt FLOAT,
    three_pt_pct DECIMAL(5,3),
    two_pt FLOAT,
    two_pt_pct DECIMAL(5,3),
    efg_pct DECIMAL(5,3),
    ft FLOAT,
    fta FLOAT,
    ft_pct DECIMAL(5,3),
    orb FLOAT,
    drb FLOAT,
    trb FLOAT,
    ast FLOAT,
    stl FLOAT,
    blk FLOAT,
    tov FLOAT,
    pf FLOAT,
    pts FLOAT,
    is_total_row BOOLEAN,
    FOREIGN KEY (playerID) REFERENCES players(playerID),
    FOREIGN KEY (seasonID) REFERENCES seasons(seasonID),
    FOREIGN KEY (teamCode) REFERENCES teams(teamCode)
);

CREATE TABLE player_awards (
    playerID INT NOT NULL,
    seasonID INT NOT NULL,
    awardName VARCHAR(30),
    awardValue INT,
    FOREIGN KEY (playerID) REFERENCES players(playerID),
    FOREIGN KEY (seasonID) REFERENCES seasons(seasonID)
);


SyntaxError: invalid syntax (912602225.py, line 1)

In [ ]:
import mysql.connector
#PLAYERS DATAFRAME FOR SQL INSERTION
all_players = set()

for year in range(2000, 2025):
    df = dfs[year]
    players = df['player'].unique()
    all_players.update(players)

playersDF = pd.DataFrame(sorted(all_players), columns=['playerName'])
playersDF = playersDF.dropna().drop_duplicates()


con = mysql.connector.connect(
    host = 'X',
    port = 0,
    user = 'X',
    password = 'X',
    database = 'X'
)

cursor = con.cursor()

insert_command = "INSERT INTO players (playerName) VALUES (%s)"

for player in playersDF['playerName']:
    cursor.execute(insert_command,(player,))

con.commit()
cursor.close()
con.close()



In [15]:
dfs[2001].head(2)

,rk,player,age,team,pos,games_played,games_started,minutes_played,fg,fga,fg_pct,three_pt,three_pt_att,three_pt_pct,two_pt,two_pt_att,two_pt_pct,efg_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,awards,season,is_total_row
0,1.0,Allen Iverson,25.0,PHI,SG,71.0,71.0,42.0,10.7,25.5,0.420,1.4,4.3,0.320,9.4,21.2,0.441,0.447,8.2,10.1,0.814,0.7,3.1,3.8,4.6,2.5,0.3,3.3,2.1,31.1,"MVP-1,DPOY-11,AS,NBA1",2000-01,False
1,2.0,Jerry Stackhouse,26.0,DET,SG,80.0,80.0,40.2,9.7,24.1,0.402,2.1,5.9,0.351,7.6,18.2,0.418,0.445,8.3,10.1,0.822,1.2,2.7,3.9,5.1,1.2,0.7,4.1,2.0,29.8,"MVP-14,AS",2000-01,False


In [ ]:
#PROGRAM USED TO INSERT INTO SEASONS TABLE

all_years = set()
for year in range(2000,2026):
    df = dfs[year]
    years = df['season'].unique()
    all_years.update(years)


con = mysql.connector.connect(
    host = 'X',
    port = 0,
    user = 'X',
    password = 'X',
    database = 'X'
)


cursor = con.cursor()

insert_seasons_command = "INSERT INTO seasons (year) VALUES (%s)"

for dates in all_years:
    cursor.execute(insert_seasons_command,(dates,))

con.commit()
cursor.close()
con.close()




In [ ]:
#PROGAM FOR TEAMS SQL INSERTION 

team_map = {
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BRK': 'Brooklyn Nets',
    'CHA': 'Charlotte Hornets',
    'CHH': 'Charlotte Hornets (old)',
    'CHI': 'Chicago Bulls',
    'CHO': 'Charlotte Hornets',  
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NOH': 'New Orleans Hornets',
    'NOK': 'New Orleans/Oklahoma City Hornets',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHO': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'SEA': 'Seattle SuperSonics',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards',
    'VAN': 'Vancouver Grizzlies',
    'NJN': 'New Jersey Nets',
    '2TM': 'Played for 2 teams',
    '3TM': 'Played for 3 teams',
    '4TM': 'Played for 4 teams',
    '5TM': 'Played for 5 teams',
    'nan': 'No team / Missing data',
}

# Franchise grouping
franchise_map = {
    'BRK': 'BRK', 'NJN': 'BRK',
    'NOP': 'NOP', 'NOH': 'NOP', 'NOK': 'NOP',
    'CHA': 'CHA', 'CHO': 'CHA', 'CHH': 'CHH',  
    'OKC': 'OKC', 'SEA': 'OKC',
    'MEM': 'MEM', 'VAN': 'MEM',
    # All others use themselves as franchiseID
}

# Add everything else that maps to itself
for code in team_map:
    if code not in franchise_map:
        franchise_map[code] = code

team_codes = [t for t in all_teams if isinstance(t, str)] 
team_list = []

for code in sorted(team_codes):
    team_list.append({
        'teamCode': code,
        'teamName': team_map.get(code, 'Unknown'),
        'franchiseID': franchise_map.get(code, 'Unknown')
    })

teams_df = pd.DataFrame(team_list)

insert_team_sql = """
INSERT INTO teams (teamCode, teamName, franchiseID)
VALUES (%s, %s, %s)
"""

con = mysql.connector.connect(
    host = 'X',
    port = 0,
    user = 'X',
    password = 'X',
    database = 'X'
)


cursor = con.cursor()

for _, row in teams_df.iterrows():
    cursor.execute(insert_team_sql, (row['teamCode'], row['teamName'], row['franchiseID']))
con.commit()


con.commit()
cursor.close()
con.close()


In [ ]:
### INSERTION INTO PLAYER STATS

con = mysql.connector.connect(
    host = 'X',
    port = 0,
    user = 'X',
    password = 'X',
    database = 'X'
)

cursor = con.cursor()


insert_stmt = """
INSERT INTO player_stats (
    playerID, seasonID, age, teamCode, pos,
    games_played, games_started, minutes_played,
    fg, fga, fg_pct,
    three_pt, three_pt_pct,
    two_pt, two_pt_pct,
    efg_pct, ft, fta, ft_pct,
    orb, drb, trb, ast, stl, blk, tov, pf, pts, is_total_row
) VALUES (
    %s, %s, %s, %s, %s,
    %s, %s, %s,
    %s, %s, %s,
    %s, %s,
    %s, %s,
    %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)
"""

# Fetch playerID and seasonID maps
cursor.execute("SELECT playerID, playerName FROM players")
player_map = {name: pid for pid, name in cursor.fetchall()}

cursor.execute("SELECT seasonID, year FROM seasons")
season_map = {year: sid for sid, year in cursor.fetchall()}


for year, df in dfs.items():
    df = df.copy()

    if 'teamCode' not in df.columns and 'team' in df.columns:
        df.rename(columns={'team': 'teamCode'}, inplace=True)

    for _, row in df.iterrows():
        playerID = player_map.get(row['player'])
        seasonID = season_map.get(row['season'])

        if not playerID or not seasonID:
            continue  

        try:
            values = (
                playerID,
                seasonID,
                int(row['age']) if pd.notnull(row['age']) else None,
                row['teamCode'],
                row['pos'],
                int(row['games_played']) if pd.notnull(row['games_played']) else None,
                int(row['games_started']) if pd.notnull(row['games_started']) else None,
                float(row['minutes_played']) if pd.notnull(row['minutes_played']) else None,
                float(row['fg']) if pd.notnull(row['fg']) else None,
                float(row['fga']) if pd.notnull(row['fga']) else None,
                float(row['fg_pct']) if pd.notnull(row['fg_pct']) else None,
                float(row['three_pt']) if pd.notnull(row['three_pt']) else None,
                float(row['three_pt_pct']) if pd.notnull(row['three_pt_pct']) else None,
                float(row['two_pt']) if pd.notnull(row['two_pt']) else None,
                float(row['two_pt_pct']) if pd.notnull(row['two_pt_pct']) else None,
                float(row['efg_pct']) if pd.notnull(row['efg_pct']) else None,
                float(row['ft']) if pd.notnull(row['ft']) else None,
                float(row['fta']) if pd.notnull(row['fta']) else None,
                float(row['ft_pct']) if pd.notnull(row['ft_pct']) else None,
                float(row['orb']) if pd.notnull(row['orb']) else None,
                float(row['drb']) if pd.notnull(row['drb']) else None,
                float(row['trb']) if pd.notnull(row['trb']) else None,
                float(row['ast']) if pd.notnull(row['ast']) else None,
                float(row['stl']) if pd.notnull(row['stl']) else None,
                float(row['blk']) if pd.notnull(row['blk']) else None,
                float(row['tov']) if pd.notnull(row['tov']) else None,
                float(row['pf']) if pd.notnull(row['pf']) else None,
                float(row['pts']) if pd.notnull(row['pts']) else None,
                bool(row['is_total_row']) if pd.notnull(row['is_total_row']) else None
            )

            cursor.execute(insert_stmt, values)

        except Exception as e:
            print(f" Error inserting row for player {row['player']} ({year}): {e}")
            continue

con.commit()
cursor.close()
con.close()



⚠️ Error inserting row for player League Average (2000): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2001): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2002): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2003): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2004): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2005): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2006): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2007): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2008): 1048 (23000): Column 'age' cannot be null
⚠️ Error inserting row for player League Average (2009): 1048 (23000): Column 'age' cannot be null
⚠️ Error i